In [1]:
# %%configure -f
# {
#     "vCores": 4, 
#     "mountPoints": [
 
#     ],
# }





In [2]:
# %%configure -f
# {
#     "vCores": 4, 
#     "defaultLakehouse": {          
#         "name": "LH_CommonCode"
#     },
#     "mountPoints": [
 
#     ],
# }

# **Librerias** 

- Se implementan los import necesario para el notebook 
- Las funciones generales se alojan sobre el Lakehouse LH_CommonCode


In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


# import sys
# sys.path.append("/lakehouse/default/Files/code_utils")
from logger_utils import log, set_logging 
from logging_utils import log_operation
from db_utils import FabricConnectionManager, create_db_connection
from sql_repository import get_connections_partner, get_connections_db
from ingestion_utils import get_schedules, get_last_watermark, get_current_watermark, get_valid_columns, get_sql_table_schema, save_data, fetch_data, fetch_data_pagination, fetch_all_data, process_platform_connection, procces_project, get_sql_table_schema_database
from format_utils import get_current_day, pandas_time_to_str, sanitize_for_pandas, format_datetime_for_sqlserver, clean_data
from partition_utils import get_block_number, get_block, get_batches, get_block_number_in_window
from concurrent.futures import ThreadPoolExecutor, as_completed
import uuid
from watermark_utils import get_all_last_watermarks, get_last_watermark_from_cache



# **Configuración logger** 

- Se permite configurar a nivel de log que se muetra durante la ejecución 


#### Mostrar solo warnings y errores

- set_logging(enabled=True, level="WARNING")

#### Activar todos los logs para debugging
set_logging(enabled=True, level="DEBUG")

#### Ejemplo de uso
log("✅ Ejemplo de uso success.", level="info") 

log("⚠️ Ejemplo de uso warning.", level="warning") 

log(f"❌ Ejemplo de uso eror: {{e}}", level="error") 

In [4]:
# Activar todos los logs para debugging
set_logging(enabled=True, level="DEBUG")


2025-10-08 19:58:00,054 - INFO - 🔧 Logging activado con nivel DEBUG


# **Main**

- Esta celda se encarga de iniciar el proceso de ingesta de los productos TryController y TryTiendas a nivel de plataforma y base de datos. Segun la franja horaria parametrizada, el sistema determina si la ejecución se realiza dentro de este horario, de ser así, identifica según la cantidad bloque configuradas que plataformas a que plataformas corresponde realizar la ingesta 

In [5]:

from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed
import gc
from deltalake import DeltaTable, write_deltalake
import time
from datetime import datetime, timedelta



PROCESS_NAME = "Data Ingestion Process"
WORK_SPACE_ID = '18536da5-57d3-408e-b9cf-94d3fdd66b8a'
WAREHOUSE_NAME = 'AnalyticsDB-OnOff'

WORK_SPACE_ID_FABRICSQL = "728e8d08-4f31-496b-992e-a925f812223c"
FABRIC_SQL_DATABASE = 'DB_Engineering'

# table_name = 'Loans'
project_name = 'TryController'

# max_workers = 4
# 11-19 
# TIME_START = 20 
# TIME_END = 5
# TOTAL_BLOCKS = 4

TIME_START = 11
TIME_END = 19
TOTAL_BLOCKS = 4
BATCH_SIZE = 70
MAX_WORKERS = 70         # procesos simultáneos
MAX_RETRIES = 3           # reintentos máximos por conexión
RETRY_WAIT = 10           # segundos entre reintentos
ENVIRONMENT = 'prod'
LOG_TABLE = 'DataOn.LastWatermarkState'
TIME_SLEEP = 20
GL_PROCESS_EXECUTION_ID = uuid.uuid4()



# opción nueva (constructor con clase)
conn_mgr_fabric = FabricConnectionManager(
    notebookutils,
    WORK_SPACE_ID_FABRICSQL,
    None,
    FABRIC_SQL_DATABASE
)


dbs = ['TryC3', 'TryC2', 'TryC0', 'TryC12', 'TryC14', 'TryC15', 'TryC17', 'TryC22', 
        'TryC48', 'TryC49', 'TryC50', 'TryC45', 'TryC64', 'Try_Tiendas_VC', 'Try_Servicios_VC']
password_central_db_conn = notebookutils.credentials.getSecret('https://kvdataon-prod.vault.azure.net/','prod-trycontroller-db1-pwd-Tcadmindb')
central_db_conn = create_db_connection("trycontroller-db1.database.windows.net", 1433, "TryController_X", "Tcadmindb", password_central_db_conn)


# Definir las tablas y sus campos primarios
df_schedules = get_schedules(conn_mgr_fabric.get_sql_connection2())


# # tablas = ["Countries", "States", "Cities", "PersonalsDataOfRoutes"]
# tablas = ['DevicesRoutesTokenSession', 'PersonalsDataOfRoutes']
# df_schedules = df_schedules[df_schedules["table_name"].isin(tablas) 
# # & (df_schedules["source_type"].str.lower() == "database")
# ]
# df_schedules = df_schedules[df_schedules['table_name'] == 'Societies']


# Identificar programaciones a ejecutar por base de datos
df_TiendasOn_db = df_schedules[(df_schedules["source_type"].str.lower() == "database") & 
    (df_schedules["project"] == "TiendasOn")] 

if 'df_TiendasOn_db' in locals() and not df_TiendasOn_db.empty:    
    resource_project = "TiendasOn"
    log(f"🏗️ Proyecto: {resource_project} | databases", level="info")
    log("🔄 Obteniendo todas las conexiones...",  level="info")
    connections = get_connections_db(project=2, engine=central_db_conn, db_list=dbs)

    grouped_by_table = defaultdict(list)
    if connections is not None:

        df_schema= get_sql_table_schema_database(connections.iloc[0],  df_TiendasOn_db)         
         
        procces_project(
            resource_project, df_TiendasOn_db, connections, df_schema, conn_mgr_fabric, PROCESS_NAME, ENVIRONMENT, LOG_TABLE, MAX_RETRIES, RETRY_WAIT, MAX_WORKERS, TIME_SLEEP, notebookutils, write_deltalake, GL_PROCESS_EXECUTION_ID
        )


    # Esperar antes de seguir con el siguiente proyecto
    log(f"⏳ Esperando antes de seguir con el siguiente proyecto...", level="info")
    time.sleep(60)


# Identificar programaciones a ejecutar por base de datos
df_TiendasOn_db = df_schedules[(df_schedules["source_type"].str.lower() == "database") & 
    (df_schedules["project"] == "TryController")] 

if 'df_TiendasOn_db' in locals() and not df_TiendasOn_db.empty:    
    resource_project = "TiendasOn"
    log(f"🏗️ Proyecto: {resource_project} | databases", level="info")
    log("🔄 Obteniendo todas las conexiones...",  level="info")
    connections = get_connections_db(project=0, engine=central_db_conn, db_list=dbs)

    grouped_by_table = defaultdict(list)
    if connections is not None:

        df_schema= get_sql_table_schema_database(connections.iloc[0],  df_TiendasOn_db)         
         
        procces_project(
            resource_project, df_TiendasOn_db, connections, df_schema, conn_mgr_fabric, PROCESS_NAME, ENVIRONMENT, LOG_TABLE, MAX_RETRIES, RETRY_WAIT, MAX_WORKERS, TIME_SLEEP, notebookutils, write_deltalake, GL_PROCESS_EXECUTION_ID
        )



    # Esperar antes de seguir con el siguiente proyecto
    log(f"⏳ Esperando antes de seguir con el siguiente proyecto...", level="info")
    time.sleep(60)



# Identificar programaciones por plataforma TiendasOn
df_pf_TiendasOn = df_schedules[(df_schedules["source_type"].str.lower() == "platform")  &
    (df_schedules["project"] == "TiendasOn")] 


resource_project = "TiendasOn"
if 'df_pf_TiendasOn' in locals() and not df_pf_TiendasOn.empty:
    log(f"🏗️ Proyecto: {resource_project}", level="info")
    log("🔄 Obteniendo todas las conexiones...",  level="info")
    df_total = get_connections_partner(2, central_db_conn)
     
    grouped_by_table = defaultdict(list)
    if df_total is not None:
        df_total.iloc[0] 

        df_schema= get_sql_table_schema(df_total.iloc[0],  df_pf_TiendasOn)  
        
        # block_number = get_block_number(TOTAL_BLOCKS)
        block_number = get_block_number_in_window(TOTAL_BLOCKS, TIME_START, TIME_END, datetime.now()- timedelta(hours=5))
        if block_number is not None:
            log(f"get_block_number_in_window {block_number}")
            df_block_conns = get_block(df_total, block_number, TOTAL_BLOCKS)
            log(f"Cantdidad de plataformas: {len(df_block_conns)}", level="info")
            procces_project (resource_project, df_pf_TiendasOn, df_block_conns, df_schema, conn_mgr_fabric, PROCESS_NAME, ENVIRONMENT, LOG_TABLE, MAX_RETRIES, RETRY_WAIT, MAX_WORKERS, TIME_SLEEP, notebookutils, write_deltalake, GL_PROCESS_EXECUTION_ID)
        else:
            log(f"No hay procesos programado en este bloque")

    # Esperar antes de seguir con el siguiente proyecto
    log(f"⏳ Esperando antes de seguir con el siguiente proyecto...", level="info")
    time.sleep(60)


# Identificar programaciones por plataforma TryController
df_pf_TryController = df_schedules[(df_schedules["source_type"].str.lower() == "platform")  &
    (df_schedules["project"] == "TryController")]
    
resource_project = "TryController"
if 'df_pf_TryController' in locals() and not df_pf_TryController.empty:
    log(f"🏗️ Proyecto: {resource_project}", level="info")
    log("🔄 Obteniendo todas las conexiones...",  level="info")
    df_total = get_connections_partner(0, central_db_conn)
    grouped_by_table = defaultdict(list)
    if df_total is not None:
        df_total.iloc[0] 

        df_schema= get_sql_table_schema(df_total.iloc[0],  df_pf_TryController)  
        
        block_number = get_block_number_in_window(TOTAL_BLOCKS, TIME_START, TIME_END, datetime.now()- timedelta(hours=5))
        if block_number is not None:
            df_block_conns = get_block(df_total, block_number, TOTAL_BLOCKS)
            log(f"Cantdidad de plataformas: {len(df_block_conns)}", level="info")
            # procces_project (resource_project, df_pf_TryController, df_block_conns, df_schema, conn_mgr_fabric, PROCESS_NAME, ENVIRONMENT, LOG_TABLE, MAX_RETRIES, RETRY_WAIT, MAX_WORKERS, TIME_SLEEP, notebookutils, write_deltalake, GL_PROCESS_EXECUTION_ID)
        else:
            log(f"No hay procesos programado en este bloque")

log(f"Ingesta finalizada en Microsoft Fabric.", level="info")


2025-10-08 19:58:01,357 - INFO - ✅ Conexion satisfactoria al servidor trycontroller-db1.database.windows.net | db TryController_X
2025-10-08 19:58:15,526 - INFO - ✅ Conexión satisfactoria | db DB_Engineering | intento 1
2025-10-08 19:58:15,531 - INFO - ✅ Nueva conexión creada para sql_DB_Engineering
2025-10-08 19:58:15,532 - INFO - ✅ Obtiene los procesos programados según el corte del día.
/lakehouse/default/Files/code_utils/ingestion_utils.py:36: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_schedules = pd.read_sql("SELECT * FROM DataOn.VwGetSchedule;", _con)
/lakehouse/default/Files/code_utils/ingestion_utils.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_keysByTa

In [7]:
# opción nueva (constructor con clase)
conn_mgr_fabric = FabricConnectionManager(
    notebookutils,
    WORK_SPACE_ID_FABRICSQL,
    None,
    FABRIC_SQL_DATABASE
)

conn_mgr_fabric.get_sql_connection2()

2025-10-08 19:18:04,763 - WARNING - ⚠️ Error en intento 1 para DB_Engineering. Esperando 5s antes de reintentar... Error: ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '<token-identified principal>'. (18456) (SQLDriverConnect)")
2025-10-08 19:18:10,129 - WARNING - ⚠️ Error en intento 2 para DB_Engineering. Esperando 10s antes de reintentar... Error: ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '<token-identified principal>'. (18456) (SQLDriverConnect)")
2025-10-08 19:18:20,503 - ERROR - ❌ Error al establecer la conexión de DB_Engineering después de 3 intentos: ('28000', "[28000] [Microsoft][ODBC Driver 18 for SQL Server][SQL Server]Login failed for user '<token-identified principal>'. (18456) (SQLDriverConnect)")
2025-10-08 19:18:20,504 - ERROR - ❌ No se pudo obtener conexión para sql_DB_Engineering. Pool lleno.


In [ ]:
from datetime import datetime, timedelta
from partition_utils import get_block_number_in_window

TOTAL_BLOCKS = 4
# password_central_db_conn = notebookutils.credentials.getSecret('https://kvdataon-prod.vault.azure.net/','prod-trycontroller-db1-pwd-Tcadmindb')
# central_db_conn = create_db_connection("trycontroller-db1.database.windows.net", 1433, "TryController_X", "Tcadmindb", password_central_db_conn)
# df_total = get_connections_partner(2, central_db_conn)

TIME_START = 11
TIME_END = 19

cases = [
    ("12:25", TOTAL_BLOCKS, TIME_START, TIME_END),
    ("14:00", TOTAL_BLOCKS, TIME_START, TIME_END),
    ("16:00", TOTAL_BLOCKS, TIME_START, TIME_END),
    ("18:00", TOTAL_BLOCKS, TIME_START, TIME_END),
    ("18:58", TOTAL_BLOCKS, TIME_START, TIME_END),
#     ("09:00", 4, 0, 8),
#     ("23:30", 4, 22, 6),
#     ("02:30", 4, 22, 6),
#     ("21:59", 4, 22, 6),
#     ("12:00", 6, 0, 0),
]

for now_s, blocks, start, end in cases:
    now = datetime.strptime(now_s, "%H:%M")
    # print(datetime.now())
    now = now.replace(year=2025, month=9, day=24)
    # print(now)
    blk = get_block_number_in_window(blocks, start, end, now=now)
    log(f"{now_s} blocks {blocks} {start:02d}:00-{end:02d}:00 -> {blk}", level="info")

    # df_block_conns = get_block(df_total, blk, TOTAL_BLOCKS)

log("Fuera", level="info")    
times = datetime.now() - timedelta(hours=5)
log(f"{times}", level="info")

blk = get_block_number_in_window(4, 9, 16, times)
log(f"{blk}", level="info")





